# Archeo



## 1. Training the model

### 1.1 Loading of paths and libraries, as well as the model configuration skeleton

In [1]:
! pip install segmentation_models_pytorch

In [2]:
#trying to use old version of lightning to get it working and will update later.
!pip install lightning==1.9
import os
import random 
import numpy as np
import pandas as pd
import rich

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations import pytorch

import lightning 
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import TQDMProgressBar 
from lightning.pytorch.callbacks import RichProgressBar

import torchvision.ops
import segmentation_models_pytorch  as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

from datetime import datetime

/Users/hatton/Nextcloud/my_files/code/adapted_tell_segmentation/casini_paper-env/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
os.getcwd()

'/Users/hatton/Nextcloud/my_files/code/adapted_tell_segmentation'

In [4]:
PATH_LOG= os.path.join(os.getcwd(),'exp_logs/')
PATH_DATASETS=os.path.join(os.getcwd(),'datasets/')

modelli=['bing_1k','bing_1k_filtrato','bing+corona_1k','bing+corona_2k_filtrato',
         'bing_2k','bing_2k_filtrato']
testset_modelli={}

config = {
    "timestamp" : datetime.now().strftime("%d-%m-%Y_%H%M%S"),
    "dataset_path" : "",
    "checkpoint_path":PATH_LOG,
    "random_seed" : 1234,
    "arch":"MAnet", #Unet,MAnet
    "encoder":"efficientnet-b3", #resnet18, dpn68, efficientnet-b3
    "weights":"imagenet",
    "loss":"focal",
    "learning_rate":0.0001,
    "precision":32,
    "epochs":5,       #was 20
    "batch_size":32,
    "corona_path":"",
    "dim_input":'',
    "in_channels":0
}

random.seed(config["random_seed"])
np.random.seed(config["random_seed"])
torch.manual_seed(config["random_seed"])

### 1.2 Division of the dataset into train set (80%), validation set (10%) and test set (10%)

In [5]:
def load_dataset(PATH,SEED,indices):
    root_directory = os.path.join(PATH)
    images_directory = os.path.join(root_directory, "train/sites")
    masks_directory = os.path.join(root_directory, "train/masks")
    
    filenames_train = np.asarray(list(sorted(os.listdir(images_directory))))
    print("total files:",len(filenames_train))

    valid_split = -int(len(indices)*0.2)
    test_split = valid_split//2
    
    train_indices = indices[:valid_split]
    valid_indices = indices[valid_split:test_split]
    test_indices = indices[test_split:]
    train_images_filenames = filenames_train[train_indices]
    val_images_filenames = filenames_train[valid_indices]
    test_images_filenames = filenames_train[test_indices]

    print("root:",root_directory,"\nimages",images_directory,
          "\nmasks",masks_directory,"\n---",
          '\ntrain images',len(train_images_filenames), 
          '\nval images',len(val_images_filenames), 
          '\ntest images',len(test_images_filenames),
          '\n---\ntotal images',len(filenames_train)
         )
    
    print("empty masks percentage: %.4f %.4f %.4f" % 
          (np.sum([i.startswith("neg") for i in train_images_filenames])/len(train_images_filenames),
            np.sum([i.startswith("neg") for i in val_images_filenames])/len(val_images_filenames),
            np.sum([i.startswith("neg") for i in test_images_filenames])/len(test_images_filenames)
          ))
    
    return images_directory, masks_directory, train_images_filenames, val_images_filenames,test_images_filenames

#### Crop and resize depend on input size 

In [6]:
def transform_images():
  if(config["dim_input"]=='1k' and config['corona_path']!=""): #same crop for input bing and input corona
    train_transform = A.Compose([
            A.RandomCrop(512,512,p=1.0),
            A.Flip(p=0.25),A.RandomRotate90(p=0.25),
            A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.25),
            A.Resize(256, 256),
            A.pytorch.ToTensorV2(),
        ],
        additional_targets={'image_corona': 'image'})
  

    val_transform = A.Compose([
            A.RandomCrop(512,512,p=1.0),
            A.Resize(256, 256),
            A.pytorch.ToTensorV2()
        ],
        additional_targets={'image_corona': 'image'})
  elif(config["dim_input"]=='1k' and config['corona_path']==""):
        train_transform = A.Compose([
            A.RandomCrop(512,512,p=1.0),
            A.Flip(p=0.25),A.RandomRotate90(p=0.25)
            ,A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.25),
            A.Resize(256, 256),
            A.pytorch.ToTensorV2(),
        ])
     
        val_transform = A.Compose([
            A.RandomCrop(512,512,p=1.0),
            A.Resize(256, 256),
            A.pytorch.ToTensorV2()
        ])

  elif(config["dim_input"]=='2k' and config['corona_path']!=""):
    train_transform = A.Compose([
            A.RandomCrop(1024,1024,p=1.0),
            A.Flip(p=0.25),A.RandomRotate90(p=0.25)
            ,A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.25),
            A.Resize(512, 512),
            A.pytorch.ToTensorV2(),
        ],
        additional_targets={'image_corona': 'image'})
  
    val_transform = A.Compose([
            A.RandomCrop(1024,1024,p=1.0),
            A.Resize(512, 512),
            A.pytorch.ToTensorV2()
        ],
        additional_targets={'image_corona': 'image'})
  else:
    train_transform = A.Compose([
            A.RandomCrop(1024,1024,p=1.0),
            A.Flip(p=0.25),A.RandomRotate90(p=0.25)
            ,A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.25),
            A.Resize(512, 512),
            A.pytorch.ToTensorV2(),
        ])
  
    val_transform = A.Compose([
            A.RandomCrop(1024,1024,p=1.0),
            A.Resize(512, 512),
            A.pytorch.ToTensorV2()
        ])
  return train_transform,val_transform

  

###  1.3 Loading the dataset with its transformations

#### Loading the dataset with its transformations, based on the type of model we are analysing,

#### based on the input size of each image and the input channels. 

In [7]:
class ArcheoDataset(Dataset):
    def __init__(self, images_filenames, images_directory, masks_directory, transform=None):
        self.images_filenames = images_filenames
        self.images_directory = images_directory
        self.masks_directory = masks_directory
        self.transform = transform

    def __len__(self):
        return len(self.images_filenames)


    def __getitem__(self, idx):
        image_filename = self.images_filenames[idx]
        image_path = os.path.join(self.images_directory, image_filename)
        mask_path = os.path.join(self.masks_directory, image_filename.replace(".jpg", ".png"))
        
        image = np.array(Image.open(image_path))#.convert("RGB"))
        image = Image.open(image_path)
      
        
        mask = ~np.array(Image.open(mask_path).convert("L")) # masks are flipped because of qgis
        mask = mask.astype("float")
        mask[mask > 0.0] = 1.0
        mask = np.expand_dims(mask, -1)


        if(config['corona_path']!=""):
          path_corona=config['corona_path']
          image_path_corona=os.path.join(path_corona, image_filename)
          image_corona =np.array(Image.open(image_path_corona))
          image_corona = Image.open(image_path_corona)
          transformed = self.transform(image=np.asarray(image),
                                       image_corona=np.asarray(image_corona),
                                       mask=np.asarray(mask))
          image_corona=transformed["image_corona"]
          image = transformed["image"]
          image=torch.cat((image, image_corona), 0)
      
        else:
          transformed = self.transform(image=np.asarray(image),mask=np.asarray(mask))
          image = transformed["image"]

        
        mask = transformed["mask"].permute(2,0,1)
        
            
        return image, mask, image_filename








### 1.4 Initialising the model

model initialisation, calculation of loss function on masks, probabilities obtained 
with sigmoid. Probabilities greater than 0.5 are transformed into 1, those below 0.5 into 0.  

In [20]:
class ArcheoModel(lightning.LightningModule):

    def __init__(self, arch, encoder_name, in_channels, out_classes, **kwargs):
        super().__init__()
        self.training_step_outputs = []
        self.model = smp.create_model(
            arch, encoder_name=encoder_name, in_channels=in_channels, classes=out_classes, **kwargs
        )
        params = smp.encoders.get_preprocessing_params(encoder_name)
        self.register_buffer("std", torch.tensor(params["std"]).view(1, 3, 1, 1))
        self.register_buffer("mean", torch.tensor(params["mean"]).view(1, 3, 1, 1))
        
        if config["loss"] == "jaccard":
            self.loss_fn = smp.losses.JaccardLoss(smp.losses.BINARY_MODE, from_logits=True)
        if config["loss"] == "dice":
            self.loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
        if config["loss"] == "focal":
            self.loss_fn = smp.losses.FocalLoss(mode=smp.losses.BINARY_MODE)
        

    def forward(self, image):
        
        if(config['corona_path']!=''):
          image1,image2=torch.tensor_split(image, 2, dim=1)
          image1 = (image1 - self.mean) / self.std
          image2 = (image2 - self.mean) / self.std
          image=torch.cat((image1, image2), 1)
        else:
          image = (image - self.mean) / self.std
        mask = self.model(image)
        return mask

    def shared_step(self, batch, stage):
        image = batch[0]
        assert image.ndim == 4
        h, w = image.shape[2:]
        assert h % 32 == 0 and w % 32 == 0
        mask = batch[1]
        assert mask.ndim == 4
        assert mask.max() <= 1.0 and mask.min() >= 0
        logits_mask = self.forward(image)
        #calculate loss
        loss = self.loss_fn(logits_mask, mask)
        #Logging loss to a dictionery
        self.log_dict({f"{stage}/loss": loss.detach().item()},batch_size=config["batch_size"])
        prob_mask = logits_mask.sigmoid()
        pred_mask = (prob_mask > 0.5).float()  
        pred_mask = pred_mask.permute(0,3,1,2)
        mask = mask.permute(0,3,1,2)
        
        tp, fp, fn, tn = smp.metrics.get_stats(pred_mask.long(), mask.long(), mode="binary")

        self.training_step_outputs.append([tp,fp,fn,tn])
        
        if stage == "train":
            self.log_dict({
                "train/batch-IOU-img":smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro-imagewise"),
                "train/batch-IOU":smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro")
            }, prog_bar=True, batch_size=config["batch_size"])

        return {
            "loss": loss,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn,
        }

    def shared_epoch_end(self, outputs, stage):
        tp = torch.cat([x["tp"] for x in outputs])
        fp = torch.cat([x["fp"] for x in outputs])
        fn = torch.cat([x["fn"] for x in outputs])
        tn = torch.cat([x["tn"] for x in outputs])

        per_image_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro-imagewise")
        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro")

        metrics = {
            f"{stage}/IOU-img": per_image_iou,
            f"{stage}/IOU": dataset_iou,
        }
        
        self.log_dict(metrics, prog_bar=True,batch_size=config["batch_size"])

    def training_step(self, batch, batch_idx):
        return self.shared_step(batch, "train")          

    #def training_epoch_end(self, outputs): #Deprecated below is new method
        #return self.shared_epoch_end(outputs, "train")

    ##New method
    #def on_train_epoch_end(self):
        #epoch_average = torch.stack

    ## New Method
    #def on_train_batch_end(self, outputs):
       # self.train_loss_metric(outputs['loss'])
       # self.log(
       #     "finetuning/train_loss",
       #     self.train_loss_metric,
       #     prog_bar=self.prog_bar,
       #     on_step=False,
       #     on_epoch=True
       # )

    def validation_step(self, batch, batch_idx):
        return self.shared_step(batch, "valid")
    
    #def validation_epoch_end(self, outputs):
        #return self.shared_epoch_end(outputs, "valid")
    
    #def on_validation_epoch_end(self, outputs):
        #return self.shared_epoch_end(outputs, "valid")
    
    def test_step(self, batch, batch_idx):
        return self.shared_step(batch, "test")  

    def test_epoch_end(self, outputs):
        return self.shared_epoch_end(outputs, "test")

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=config["learning_rate"]) # 0.00005


In [21]:
def set_config(name_modello):
  config["dataset_path"]=PATH_DATASETS+name_modello
  if '1k' in name_modello:
    config["dim_input"]='1k'
    if('+' in name_modello):
      config["dataset_path"]=PATH_DATASETS+'bing_1k/'
      config['corona_path']=PATH_DATASETS+'corona_1k/train/sites/'
      config['in_channels']=6
    else:
      config['corona_path']=""
      config['in_channels']=3
  else:
    config["dim_input"]='2k'
    if('+' in name_modello):
      config['corona_path']=PATH_DATASETS+'corona_2k_filtrato/train/sites/'
      config['in_channels']=6
      config["dataset_path"]=PATH_DATASETS+'bing_2k_filtrato/'
    else:
      config['corona_path']=""
      config['in_channels']=3

### 1.5 Training of each model according to its characteristics

In [22]:
 for i in range(len(modelli)):
  set_config(modelli[i])
  filenames_train = np.asarray(list(sorted(os.listdir(os.path.join(config["dataset_path"], "train/sites")))))
  print("total files:",len(filenames_train))
  indices = np.arange(0,len(filenames_train))
  np.random.shuffle(indices)
  print(indices)
  

  images_directory, masks_directory, train_images_filenames,val_images_filenames,test_images_filenames = load_dataset(config["dataset_path"],config["random_seed"], indices)
  train_transform,val_transform=transform_images()
  train_dataset = ArcheoDataset(train_images_filenames, images_directory, 
                                masks_directory,transform=train_transform)
  val_dataset = ArcheoDataset(val_images_filenames, images_directory, 
                              masks_directory, transform=val_transform)
  train_loader = DataLoader(train_dataset,batch_size=config["batch_size"], 
                            shuffle=True,drop_last=True,num_workers=0,)
  val_loader = DataLoader(val_dataset,batch_size=config["batch_size"], 
                          shuffle=False,drop_last=False,num_workers=0,)
  model = ArcheoModel(config["arch"],encoder_name=config["encoder"],
                      encoder_weights=config["weights"],
                      in_channels=config['in_channels'], 
                      out_classes=1,) #creazione del modello

#TRAINING DEL MODELLO

  trainer = lightning.Trainer(
    max_epochs=config["epochs"],
    precision=config["precision"],
    accelerator="cpu",
    logger=pl_loggers.TensorBoardLogger(config["checkpoint_path"]),
    log_every_n_steps=1,
    enable_progress_bar=True,
    callbacks=[RichProgressBar(refresh_rate=1)],
   
  )
  cfg_text = "\n".join([ str(key)+" : **"+str(config[key])+"**  " for key in config])
  print(cfg_text)
  trainer.logger.experiment.add_text(tag="config",text_string=cfg_text)

  trainer.fit(
    model,
    train_dataloaders=train_loader, 
    val_dataloaders=val_loader)
  os.rename(PATH_LOG+'lightning_logs/version_0', PATH_LOG+'lightning_logs/'+modelli[i])




GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


total files: 4734
[1933  972  173 ... 2128 1130 3216]
total files: 4734
root: /Users/hatton/Nextcloud/my_files/code/adapted_tell_segmentation/datasets/bing_1k 
images /Users/hatton/Nextcloud/my_files/code/adapted_tell_segmentation/datasets/bing_1k/train/sites 
masks /Users/hatton/Nextcloud/my_files/code/adapted_tell_segmentation/datasets/bing_1k/train/masks 
--- 
train images 3788 
val images 473 
test images 473 
---
total images 4734
empty masks percentage: 0.0000 0.0000 0.0000
timestamp : **10-10-2024_203249**  
dataset_path : **/Users/hatton/Nextcloud/my_files/code/adapted_tell_segmentation/datasets/bing_1k**  
checkpoint_path : **/Users/hatton/Nextcloud/my_files/code/adapted_tell_segmentation/exp_logs/**  
random_seed : **1234**  
arch : **MAnet**  
encoder : **efficientnet-b3**  
weights : **imagenet**  
loss : **focal**  
learning_rate : **0.0001**  
precision : **32**  
epochs : **5**  
batch_size : **32**  
corona_path : ****  
dim_input : **1k**  
in_channels : **3**  


┏━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name    ┃ Type      ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ model   │ MAnet     │ 17.2 M │ train │
│ 1 │ loss_fn │ FocalLoss │      0 │ train │
└───┴─────────┴───────────┴────────┴───────┘

Trainable params: 17.2 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 17.2 M                                                                                               
Total estimated model params size (MB): 68                                                                         
Modules in train mode: 539                                                                                         
Modules in eval mode: 0

Output()

/Users/hatton/Nextcloud/my_files/code/adapted_tell_segmentation/casini_paper-env/lib/python3.12/site-packages/light
ning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may 
be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the 
`DataLoader` to improve performance.

`Trainer.fit` stopped: `max_epochs=5` reached.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/hatton/Nextcloud/my_files/code/adapted_tell_segmentation/datasets/bing_1k_filtrato/train/sites'

## 2 Testing of each model

In [14]:
from pytorch_lightning.loggers import tensorboard
%reload_ext tensorboard
%tensorboard --logdir=PATH_LOG

In [45]:
#10 tests, with different transformations, for each model. 
iou_test_modelli={}
for i in range(len(modelli)):
  set_config(modelli[i])
 
  filenames_train = np.asarray(list(sorted(os.listdir(
      os.path.join(config["dataset_path"], "train/sites")))))
  print("total files:",len(filenames_train))
  indices = np.arange(0,len(filenames_train))
  np.random.shuffle(indices)
  print(indices)
  
  
 
  name_ckpt = os.listdir(PATH_LOG+'lightning_logs/'+modelli[i]+'/checkpoints/')[0]
  model = ArcheoModel.load_from_checkpoint(arch=config["arch"],
                 encoder_name=config["encoder"], 
                 encoder_weights=config["weights"],
                 in_channels=config['in_channels'], 
                 out_classes=1,
                 checkpoint_path=PATH_LOG+'lightning_logs/'+modelli[i]+'/checkpoints/'+name_ckpt)
  print(config['dataset_path'])
  images_directory, masks_directory, train_images_filenames,
  val_images_filenames,test_images_filenames = load_dataset(config["dataset_path"],
                                                            config["random_seed"],indices)
  
  testiou=[]
  for j in range(10):
    print("Risultato "+str(j)+" del modello "+modelli[i])
    train_transform,val_transform=transform_images()
    test_dataset = ArcheoDataset(test_images_filenames, images_directory, 
                                 masks_directory, transform=val_transform)
    test_loader = DataLoader(test_dataset,batch_size=config["batch_size"],
                             shuffle=False,drop_last=False,num_workers=0,)

    trainer = L.Trainer(gpus=1, max_epochs=40,auto_scale_batch_size="binsearch",
                         precision=16,accelerator="auto",)
 
    test_metrics = trainer.test(model, dataloaders=test_loader, verbose=True)
    testiou.append(test_metrics[0]['test/IOU-img'])
  iou_test_modelli[modelli[i]]=testiou

total files: 4734
[4066 1623 1403 ... 3970  278 1539]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/hatton/Nextcloud/My files/code/casini_2023/adapted_tell_segmentation/exp_logs/lightning_logs/bing_1k/checkpoints/'

In [ ]:
#Stampo le statistiche per ogni modello (mean,min,max,std)
for i in modelli:
  x=np.array(iou_test_modelli[i])
  print("Le statistiche sul test set per il modello "+i+" sono:")
  print("media:"+str(round(x.mean(),4))+" | min:"+str(round(x.min(),4))
    +" | max:"+str(round(x.max(),4))+" | std:"+str(round(x.std(),4)))

Le statistiche sul test set per il modello bing_1k sono:
media:0.7417 | min:0.7356 | max:0.7468 | std:0.0038
Le statistiche sul test set per il modello bing_1k_filtrato sono:
media:0.781 | min:0.7689 | max:0.788 | std:0.0054
Le statistiche sul test set per il modello bing+corona_1k sono:
media:0.7406 | min:0.7347 | max:0.746 | std:0.0039
Le statistiche sul test set per il modello bing+corona_2k_filtrato sono:
media:0.8345 | min:0.8312 | max:0.8376 | std:0.0018
Le statistiche sul test set per il modello bing_2k sono:
media:0.7977 | min:0.7929 | max:0.8031 | std:0.0034
Le statistiche sul test set per il modello bing_2k_filtrato sono:
media:0.8154 | min:0.8087 | max:0.8223 | std:0.0035


##3 Valutazione Modelli

### 3.1 Caricamento dei csv contenenti coordinate siti

In [ ]:
import pandas as pd
import numpy as np
import csv
import seaborn as sns
import json


sns.set_style("whitegrid")
namesite2Centroid={}
nameneg2Centroid={}
namemaysan2Centroid={}
name2min={}


with open('trainset1000.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      namesite2Centroid[row[2]]=[row[3],row[4]]
      

with open('negs1000.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        nameneg2Centroid[row[5]]=[row[3],row[4]]


with open('maysan1000.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        namemaysan2Centroid[row[1]]=[row[2],row[3]]



###3.2 Carico il dataset applicandogli trasformazioni prestabilite





In [ ]:
class ArcheoDatasetModify(Dataset):
    def __init__(self, images_filenames, images_directory, masks_directory):
        self.images_filenames = images_filenames
        self.images_directory = images_directory
        self.masks_directory = masks_directory
     
    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, idx, verbose=False):
      image_filename = self.images_filenames[idx]+'.jpg'
      image_path = os.path.join(self.images_directory, image_filename)
      mask_path = os.path.join(self.masks_directory, 
                               image_filename.replace(".jpg", ".png"))
        
      image = np.array(Image.open(image_path))#.convert("RGB"))
      image = Image.open(image_path)
      # masks are flipped because of qgis
      mask = ~np.array(Image.open(mask_path).convert("L")) 
      mask = mask.astype("float")
      mask[mask > 0.0] = 1.0
      mask = np.expand_dims(mask, -1)
      x_minore=int(name2tras[image_filename[:-4]][0])
      y_minore=int(name2tras[image_filename[:-4]][1])
      if(config['corona_path']!=''):     
        path_corona=config['corona_path']
        image_path_corona=os.path.join(path_corona, image_filename)
        image_corona =np.array(Image.open(image_path_corona))
        image_corona = Image.open(image_path_corona)
  
        trasformazione = A.Compose([
            A.Crop(x_min=x_minore, y_min=y_minore, 
                   x_max=x_minore+1024, y_max=y_minore+1024,p=1.0),
            A.Resize(512,512),
            A.pytorch.ToTensorV2()],additional_targets={'image_corona': 'image'})
        transformed = trasformazione(image=np.asarray(image),
                                     image_corona=np.asarray(image_corona),
                                     mask=np.asarray(mask))
        image_corona=transformed["image_corona"]
        image = transformed["image"]
        image=torch.cat((image, image_corona), 0)
      
      else:
        trasformazione = A.Compose([
            A.Crop(x_min=x_minore, y_min=y_minore, 
                   x_max=x_minore+1024, y_max=y_minore+1024,p=1.0),
            A.Resize(512,512),
            A.pytorch.ToTensorV2()])
        transformed = trasformazione(image=np.asarray(image),
                                     mask=np.asarray(mask))
        image = transformed["image"]

      mask = transformed["mask"].permute(2,0,1)
      return image, mask, image_filename

###3.3 Salvataggio delle singole predizioni

In [ ]:
from scipy.ndimage import gaussian_filter
import cv2
import numpy as np
from matplotlib import pyplot as plt

#data la predizione,viene applicato un cutoff
def stampa_predizioni_cutoff(ibatch,ipr_masks_11,num,nome_modello,val):
    for i,(image, gt_mask, masks_11, fn) in enumerate(zip(ibatch[0], ibatch[1], 
                                                          ipr_masks_11, ibatch[2])):
      masks_11 = masks_11.numpy().squeeze()
      # cutoff
      cf = masks_11.copy()
      cf = gaussian_filter(cf, sigma=5)
      cf = ((cf + 0.5)**2) - 0.5
      cf[cf<=val] = 0.0
      cf[cf>val] = 1.0


      plt.imsave('/output/'+nome_modello+'/pred_siti_tronc'+
                 str(val)+'/'+ fn[:-4]+'.png',cf, cmap="magma", vmin=0.0, vmax=1.0)
      plt.show()

###3.4 Date le predizioni per ogni sito, creo i tif e gli shapefile corrispondenti

In [ ]:
#Dato la predizione, viene creato il tif associato. 
import rasterio
def createTif(ovest,sud,est,nord,patin,patout):
  dataset = rasterio.open(patin, 'r')
  bands = [1, 2, 3]
  data = dataset.read(bands)
  transform = rasterio.transform.from_bounds(ovest, sud, est, nord, data.shape[2], data.shape[1])
  crs = {'init': 'epsg:3857'}

  with rasterio.open(patout, 'w', driver='GTiff',
                   width=data.shape[2], height=data.shape[1],
                   count=3, dtype=data.dtype, nodata=0,
                   transform=transform, crs=crs) as dst:
      dst.write(data, indexes=bands)

In [ ]:
#Ricostruisco la predizione usando le trasformazioni salvate in precedenza.
def convertImageToTif(nome_modello,val_tronc,path_pred,path_pred_total,path_tif):
  for i in range(len(test_images_filenames)):
    
    im_sfondo=Image.open('black_2048.jpg')
    im_sito = Image.open(path_pred+test_images_filenames[i]+'.png')
    
    newsize = (1024, 1024)
    im_sito = im_sito.resize(newsize)
    x_min=int(name2tras[test_images_filenames[i]][0])
    x_max=(int(name2tras[test_images_filenames[i]][0])+1024)
    y_min=int(name2tras[test_images_filenames[i]][1])
    y_max=(int(name2tras[test_images_filenames[i]][1])+1024)
    im_sfondo.paste(im_sito, (x_min, y_min,x_max,y_max))
    im_sfondo = im_sfondo.save(path_pred_total+test_images_filenames[i]+'.png')
    if("neg" in test_images_filenames[i]):
      x_centroide=float(nameneg2Centroid[test_images_filenames[i]][0])
      y_centroide=float(nameneg2Centroid[test_images_filenames[i]][1])
    else:
      x_centroide=float(namesite2Centroid[test_images_filenames[i]][0])
      y_centroide=float(namesite2Centroid[test_images_filenames[i]][1])
    ovest=x_centroide-1000
    est=x_centroide+1000
    sud=y_centroide-1000
    nord=y_centroide+1000
    createTif(ovest=ovest, sud=sud, est=est, nord=nord,
              patin=path_pred_total+test_images_filenames[i]+'.png',
              patout=path_tif+test_images_filenames[i]+".tif")



In [ ]:
#prende i tif files e restituisce gli shape files
import os
import subprocess
from tqdm.auto import tqdm
from gdal import gdal_contour

def convertTifToShape(tif_path,shape_path):
  filenames = os.listdir(tif_path)
  print(len(filenames)) #521
  for f in tqdm(filenames):
    print(f[:-4])
    subprocess.run(["gdal_contour.exe","-i","128","-p",tif_path+f,shape_path+f[:-4]+".shp"],
                   capture_output=True,shell=True,check=False)

In [ ]:
#prende in input il file csv con le trasformazioni per ogni sito, 
#ritorna i nomi dei siti e un dizionario nome_sito -> trasformazioni
def crea_trasformazione(path_csv):
  test_sites=[]
  name2min={}
  count=0
  with open(path_csv, 'r') as file:
    reader = csv.reader(file)
    for row in (reader):
        if(count!=0):
          test_sites.append(row[1])
          name2min[row[1]]=[row[2],row[3]]
        count+=1
    
  return test_sites,name2min

###3.5 Dati gli shapefile in input e restituisco un geojson contenente per ogni sito TP,TN,FP,FN

In [ ]:
import pandas as pd
import os
import geopandas as geopd
from tqdm.auto import tqdm
from shapely.geometry.multipolygon import MultiPolygon

#Assegna ad ogni sito tp,tn,fp,fn in base all'intersezione della forma predetta 
#con la forma originale
def test_for_intersection(site_id,path,verbose=False):
    ### get shape for the original site
    sites = geopd.read_file(PATH_SHAPE).to_crs("EPSG:3857") # project to web-mercator
    a = sites[sites.entry_id == site_id][["entry_id","geometry"]]
    
    if verbose: print('Loading Contours')
    b = geopd.read_file(path+site_id+".shp").set_crs("EPSG:3857")
    b.geometry = b.geometry.convex_hull
    b["entry_id"] = "pred"
    
    if verbose: 
        print("number of features:",len(b))
        print(b)
        
    if len(b) > 1: # if there is more than one shape
        b["geometry"] = MultiPolygon([feature for feature in b["geometry"]])
        b = b[:1].copy()
        if verbose: 
            print(b)
            b.plot()
    # negs should have no geometry
    if len(b) == 0 and site_id.startswith("neg"):
        if verbose: print("good neg")
        return "TN",site_id,None
    
    # if neg has geometry then FP
    elif len(b) > 0 and site_id.startswith("neg"):
        if verbose: print("false positive")
        return "FP",site_id,b.iloc[0]["geometry"] #False
    
    # if no geometry and not neg then FN. use a.geometry for use in QGIS
    elif len(b) == 0 and not site_id.startswith("neg"):
        if verbose: print("false negative")
        return "FN",site_id, a.iloc[0]["geometry"]
    
    # compute intersection
    if ~b.iloc[0].geometry.is_valid:
        b.geometry = b.geometry.buffer(0)
    intersects = a.iloc[0]["geometry"].intersects(b.iloc[0]["geometry"])
    if verbose:
        c = pd.concat([a,b])
        # print(c)
        c.plot(column="entry_id",legend=True,figsize=(5,5),cmap="Set3")
        print("INTERSECTION: ", intersects)
        
    if intersects: # right geometry
        return "TP",site_id,b.iloc[0]["geometry"]
    else: # wrong geometry
        return "FP",site_id,b.iloc[0]["geometry"]

In [ ]:
import geopandas as geopd
#prende in input gli shapefile dei siti e ritorna un geojson 
#contenente id,nome sito, geometria e valore tra tp,tn,fp,fn
def convertShapeToGeojson(testset,path_in,path_out):
 res = {"index":[],"entry_id":[],"geometry":[],"cat":[],}
 indice=0
 for sito in testset:
    cat,eid,geom = test_for_intersection(sito,path_in,verbose=False)
    res["index"].append(indice)
    res["entry_id"].append(eid)
    res["geometry"].append(geom)
    res["cat"].append(cat)
    indice+=1
 res_df = geopd.GeoDataFrame(res)
 res_df = res_df.set_index("index")
 res_df.to_file(path_out, driver='GeoJSON',crs="EPSG:3857")  

###3.6 Main

In [ ]:
modelli_da_comparare=['bing+corona_2k_filtrato','bing_2k_filtrato']
PATH_SHAPE='/shapefiles/site_shape/vw_site_survey_poly.shp'
for modello in modelli_da_comparare:
  PATH_OUTPUT='output/'+modello+'/'
  set_config(modello)
  name_ckpt = os.listdir(PATH_LOG+'lightning_logs/'+modello+'/checkpoints/')[0]
  model= ArcheoModel.load_from_checkpoint(arch=config["arch"],
                 encoder_name=config["encoder"], 
                 encoder_weights=config["weights"],
                 in_channels=config['in_channels'], out_classes=1,
                 checkpoint_path=PATH_LOG+'lightning_logs/'+modello+'/checkpoints/'+name_ckpt)
  random.seed(config["random_seed"])
  np.random.seed(config["random_seed"])
  torch.manual_seed(config["random_seed"])
  test_images_filenames,name2tras=crea_trasformazione('trasformazioni_modello.csv')
  images_directory=config["dataset_path"]+'/train/sites'
  masks_directory=config["dataset_path"]+'/train/masks'
  test_dataset = ArcheoDatasetModify(test_images_filenames, 
                                     images_directory, 
                                     masks_directory)
  test_loader = DataLoader(test_dataset,batch_size=config["batch_size"],
                           shuffle=False,drop_last=False,num_workers=0,)

  print(len(test_images_filenames))
  it = iter(test_loader)


  for j in range(len(it)):
    batch = next(it)
    with torch.no_grad():
      model.eval()
      logits = model(batch[0])
    pr_masks = logits.sigmoid()
    stampa_predizioni_cutoff(batch,pr_masks,j,modello,0.2)
    stampa_predizioni_cutoff(batch,pr_masks,j,modello,0.5)

  convertImageToTif(modello,0.2,PATH_OUTPUT+'pred_siti_tronc0.2/',
                    PATH_OUTPUT+'pred_siti_centro_tronc0.2/',PATH_OUTPUT+'tif_0.2/')
  convertImageToTif(modello,0.5,PATH_OUTPUT+'pred_siti_tronc0.5/',
                    PATH_OUTPUT+'pred_siti_centro_tronc0.5/',PATH_OUTPUT+'tif_0.5/')

  convertTifToShape(PATH_OUTPUT+'tif_0.2/',PATH_OUTPUT+'shape_0.2/')
  convertTifToShape(PATH_OUTPUT+'tif_0.5/',PATH_OUTPUT+'shape_0.5/')

  convertShapeToGeojson(test_images_filenames,PATH_OUTPUT+'shape_0.2/',PATH_OUTPUT+'preds02.geojson')
  convertShapeToGeojson(test_images_filenames,PATH_OUTPUT+'shape_0.5/',PATH_OUTPUT+'preds05.geojson')





## 4 Analisi Risultati

### 4.1 Calcolo della matrice di confusione, in modo automatico e adattato

In [ ]:
def calcola_matrix(preds):
  tp= preds[preds.cat == "TP"]['entry_id'].shape[0]
  tn= preds[preds.cat == "TN"]['entry_id'].shape[0]
  fp= preds[preds.cat == "FP"]['entry_id'].shape[0]
  fn= preds[preds.cat == "FN"]['entry_id'].shape[0]

  matrix=[tp,tn,fp,fn]

  # sono siti non visibili, quindi classificati erroneamente come tn
  fn2tn = preds[(preds.cat == "FN")&(preds.correction == "TN")]['entry_id'].shape[0]

  sites_inside_ot=preds[(preds.notes.str.contains('INSIDE OTHER',na=False)) ].shape[0]

  #siti non visibili e il modello ne trova un altro esistente
  fp2tp = preds[
    (preds.cat == "FP")& ((preds.notes.str.contains('NV',na=False))| 
                        (preds.notes.str.contains('NOT VISIBLE',na=False)) |
                        (preds.notes.str.contains('NOT VISIBILE',na=False)) |
                        (preds.entry_id.str.contains('neg',na=False)))&
                        (preds.notes.str.contains('INSIDE OTHER',na=False))].shape[0]

  #siti visibili e il modello ne trova un altro esistente 
  fp2fn=sites_inside_ot-fp2tp

  tn_a = tn + fn2tn + fp2tp
  fn_a = fn - fn2tn + fp2fn
  tp_a = tp + (fp2tp+fp2fn)
  fp_a = fp - (fp2tp+fp2fn)

  matrix_adj=[tp_a,tn_a,fp_a,fn_a]
  
  return(matrix,matrix_adj)



In [ ]:
#mi stampo le statistiche in termini di valori della matrice di confusione oltre a
# precisione,recall.

import seaborn as sns
import matplotlib.pyplot as plt
def stampa_stats(cm,cm_adj,bing):
  print('--------------------------------------------')
  if(bing):
    print("Stats Modello Bing 2k filtrato:")
  else:
    print("Stats Modello Bing + Corona 2k filtrato:")

  print('---')

  print("Valutazione automatica:")
  print("TP: "+str(cm[0])+" TN: "+str(cm[1])+" FP: "+str(cm[2])+" FN: "+str(cm[3]))
  print("Accuracy: ",round((cm[0]+cm[1])/(cm[0]+cm[1]+cm[2]+cm[3]),4))
  print("Recall: ",round(cm[0]/(cm[0]+cm[3]),4))

  print('---')

  print("Valutazione manuale:")
  print("TP: "+str(cm_adj[0])+" TN: "+str(cm_adj[1])+" FP: "+str(cm_adj[2])+" FN: "+str(cm_adj[3]))
  print("Accuracy: ",round((cm_adj[0]+cm_adj[1])/(cm_adj[0]+cm_adj[1]+cm_adj[2]+cm_adj[3]),4))
  print("Recall: ",round(cm_adj[0]/(cm_adj[0]+cm_adj[3]),4))

  print('--------------------------------------------')

In [ ]:

import geopandas as geopd
PATH_GEOJSON_BING='/output/bing_2k_filtrato/preds05.geojson'
PATH_GEOJSON_CORONA='/output/bing+corona_2k_filtrato/preds05.geojson'
preds_bing = geopd.read_file(PATH_GEOJSON_BING).sort_values("index").reset_index(drop=True)
preds_corona = geopd.read_file(PATH_GEOJSON_CORONA).sort_values("index").reset_index(drop=True)


cm_bing,cm_bing_adj=calcola_matrix(preds_bing)
stampa_stats(cm_bing,cm_bing_adj,bing=True)


cm_corona,cm_corona_adj=calcola_matrix(preds_corona)
stampa_stats(cm_corona,cm_corona_adj,bing=False)

--------------------------------------------
Stats Modello Bing 2k filtrato:
---
Valutazione automatica:
TP: 228 TN: 98 FP: 70 FN: 125
Accuracy:  0.6257
Recall:  0.6459
---
Valutazione manuale:
TP: 258 TN: 185 FP: 40 FN: 68
Accuracy:  0.804
Recall:  0.7914
--------------------------------------------
--------------------------------------------
Stats Modello Bing + Corona 2k filtrato:
---
Valutazione automatica:
TP: 209 TN: 104 FP: 57 FN: 151
Accuracy:  0.6008
Recall:  0.5806
---
Valutazione manuale:
TP: 239 TN: 197 FP: 27 FN: 88
Accuracy:  0.7913
Recall:  0.7309
--------------------------------------------


##5 Area di selezione

###5.1 Crea le predizioni per ogni tessera

In [ ]:
PATH_MAYSAN_DATASET='/datasets/maysan_sel_tile/'
PATH_MAYSAN_OUTPUT='/output/maysan_sel_area/'
PATH_MODEL_USED=PATH_LOG+'lightning_logs/bing+corona_2k_filtrato/checkpoints/epoch=19-step=2600.ckpt'

In [ ]:
def stampa_predizioni(ibatch,ipr_masks_11,num,patout):
  for i,(image, gt_mask, masks_11, fn) in enumerate(zip(ibatch[0], ibatch[1], ipr_masks_11, ibatch[2])):

    masks_11 = masks_11.numpy().squeeze()
    plt.imsave(patout+'pred_magma_v1/'+ fn[:-4]+'.png',masks_11,cmap="magma", vmin=0.0, vmax=1.0)
    plt.imsave(patout+'pred_grey_v1/'+ fn[:-4]+'.png',masks_11,cmap="Greys", vmin=0.0, vmax=1.0)
   

In [ ]:
filenames_test_maysan = np.asarray(list(sorted(os.listdir(PATH_MAYSAN_DATASET+'sites'))))
config['corona_path']=PATH_MAYSAN_DATASET+'corona_v1/'
transform_maysan=A.Compose([A.Resize(512, 512),A.pytorch.ToTensorV2(),],
                           additional_targets={'image_corona': 'image'})
test_dataset_maysan = ArcheoDataset(filenames_test_maysan, PATH_MAYSAN_DATASET+'sites/', 
                                    PATH_MAYSAN_DATASET+'masks/', transform=transform_maysan)
test_loader_maysan = DataLoader(test_dataset_maysan,batch_size=config["batch_size"], 
                                shuffle=False,drop_last=False,num_workers=0,)
model = ArcheoModel.load_from_checkpoint(arch=config["arch"],
                 encoder_name=config["encoder"], 
                 encoder_weights=config["weights"],
                 in_channels=6, out_classes=1,checkpoint_path=PATH_MODEL_USED)

random.seed(config["random_seed"])
np.random.seed(config["random_seed"])
torch.manual_seed(config["random_seed"])

it_maysan=iter(test_loader_maysan)

from matplotlib import pyplot as plt
for i in range(len(it_maysan)):
  batch = next(it_maysan)
  with torch.no_grad():
    model.eval()
    logits = model(batch[0])
  pr_masks = logits.sigmoid()
  stampa_predizioni(batch,pr_masks,i,PATH_MAYSAN_OUTPUT)


###5.2 Assembla l'immagine totale dell'area di selezione

In [ ]:
def crea_righe(imm_size,l_x,l_y,fn,resize=False,num_res=256,format=".png"):
  righe=[]
  altezza=1
  if(resize==True):
    image_tot = Image.open(fn+"1"+format).resize((num_res,num_res))
  else:
    image_tot = Image.open(fn+"1"+format)
  for i in range(2,l_x*l_y+1):
    if((i-1)%l_x==0):
      if(resize==True):
        image_tot=Image.open(fn+str(i)+format).resize((num_res,num_res))
      else:
        image_tot=Image.open(fn+str(i)+format)
    else:
      if(resize==True):
        image_open=Image.open(fn+str(i)+format).resize((num_res,num_res))
      else:
        image_open=Image.open(fn+str(i)+format)
      new_image = Image.new('RGB',(imm_size*l_x, imm_size))
      new_image.paste(image_tot,(0,0))
      posizione=i%l_x
      if(posizione==0):
        posizione=l_x
      new_image.paste(image_open,(imm_size*(posizione-1),((altezza-1)*image_open.size[1])))
      image_tot=new_image
      if(posizione==l_x):righe.append(image_tot)
  return righe

def crea_immagine(righ,dim,l_x,l_y,nome):
  imm_totale = Image.new('RGB',(l_x*dim, l_y*dim))
  x=-dim
  for i in reversed(range(len(righ))):
    x+=dim
    imm_totale.paste(righ[i],(0,x))
  imm_totale.save(PATH_MAYSAN_OUTPUT+nome+'.jpg')

In [ ]:
import pandas as pd
import numpy as np
import csv
import seaborn as sns
import json

f = open('coor_maysan_1000.json')
data = json.load(f)
spost=data[1]['cx']-data[0]['cx']
ovest=int(data[0]['cx']-spost/2)
sud=int(data[0]['cy']-spost/2)
nord=int(data[len(data)-1]['cy']+(spost/2))
num_righe=int((nord-sud)/spost)
num_colonne=int(len(data)/num_righe)
est=int(data[num_colonne-1]['cx']+spost/2)


righe=crea_righe(512,num_colonne,num_righe,PATH_MAYSAN_OUTPUT+'pred_magma_v1/')
righe1=crea_righe(512,num_colonne,num_righe,PATH_MAYSAN_OUTPUT+'pred_grey_v1/')
crea_immagine(righe,512,num_colonne,num_righe,"imm_tot_magma_v1")
crea_immagine(righe1,512,num_colonne,num_righe,"imm_tot_grey_v1")
createTif(ovest,sud,est,nord,PATH_MAYSAN_OUTPUT+"imm_tot_magma_v1.jpg",
          PATH_MAYSAN_OUTPUT+"imm_tot_magma_v1.tif")
createTif(ovest,sud,est,nord,PATH_MAYSAN_OUTPUT+"imm_tot_grey_v1.jpg",
          PATH_MAYSAN_OUTPUT+"imm_tot_grey_v1.tif")

